In [1]:
#Space variables
cs_minX = 0 #Minimum zero
cs_maxX = 100
cs_minY = 0 #Minimum zero
cs_maxY = 100
#Numerical constants
N_turns = 500 #Total turns in the run
N_food = 200 #Shut off food
N_food2 = 600 #Turn on food
#N_food3 = 200 #No change
#N_food4 = 1000 #Shut down
N_vacc = 0

In [2]:
class FSM:
	def __init__(self,x=0,y=0):
		self.current_state = '00'
		self.current_output = '1'
		self.pos_x = x
		self.pos_y = y
	def act(self,input):
		if(input == '0' and self.current_state == '00'):
			self.current_state = '11'
			self.current_output = '1'
		elif (input == '1' and self.current_state == '00'):
			self.current_state = '01'
			self.current_output = '1'
		elif (input == '0' and self.current_state == '01'):
			self.current_state = '00'
			self.current_output = '1'
		elif (input == '1' and self.current_state == '01'):
			self.current_state = '10'
			self.current_output = '1'
		elif (input == '0' and self.current_state == '10'):
			self.current_state = '00'
			self.current_output = '1'
		elif (input == '1' and self.current_state == '10'):
			self.current_state = '01'
			self.current_output = '1'
		elif (input == '0' and self.current_state == '11'):
			self.current_state = '11'
			self.current_output = '0'
		elif (input == '1' and self.current_state == '11'):
			self.current_state = '01'
			self.current_output = '1'
	def get_state(self):
		return self.current_state
	def get_output(self):
		return self.current_output

In [3]:
class Vaccine():
	
	def __init__(self,positionData):
		self.positionSet = set()
		for l in positionData:
			self.positionSet.add(l)

	def is_vaccinated(self,posx=0,posy=0):
		return ((posx,posy) in self.positionSet)

In [4]:
from numpy import array,ones,zeros
import random as r
#import constants
#import vaccine

class CoordinateSystem:
	def __init__(
		self,
		min_x=cs_minX,
		max_x=cs_maxX,
		min_y=cs_minY,
		max_y=cs_maxY):
		self.minX = int(min_x)
		self.maxX = int(max_x)
		self.minY = int(min_y)
		self.maxY = int(max_y)
		self.space = ones((self.maxX,self.maxY))
	def move(self,fsm):
		#Move up, down, left, or right with equal probability
		deltax = 0
		deltay = 0
		if(r.randint(0,1)): #moving in x-dir
			if(r.randint(0,1)):
				deltax = 1
			else:
				deltax = -1
		else: #moving in the y-dir
			if(r.randint(0,1)):
				deltay = 1
			else:
				deltay = -1
		fsm.pos_x += deltax
		fsm.pos_y += deltay
		#Restrict movement to board...minimums typically 0
		if(fsm.pos_x>=cs_maxX):
			fsm.pos_x = cs_maxX-1
		if(fsm.pos_y>=cs_maxY):
			fsm.pos_y = cs_maxY-1
		if(fsm.pos_x<cs_minX):
			fsm.pos_x = cs_minX
		if(fsm.pos_y<cs_minY):
			fsm.pos_y = cs_minY
	def food(self,x=0,y=0,val=0):
		self.space[x,y] += val
		if(self.space[x,y]<0):
			self.space[x,y]=0
	def is_food(self,x,y,vaccine):
		if(self.space[x,y]>0 and vaccine.is_vaccinated(x,y)==False):
			return '1'
		else:
			return '0'
	def generate_food(self):
		for x in range(self.minX,self.maxX):
			for y in range(self.minY,self.maxY):
				self.food(x,y,r.randint(-1,2))
	def remove_food(self):
		for x in range(self.minX,self.maxX):
			for y in range(self.minY,self.maxY):
				self.food(x,y,r.randint(-2,1))

In [5]:
#import fsm
#import coordinate_system
#import constants

class Operate():
	def __init__(self,cs,fsm_list,vaccine):
		self.main_coordinate_system = cs
		self.main_fsm_list = fsm_list
		self.new_fsm_list = []
		self.turn_count = 0
		self.vaccine = vaccine
	def run(self):
		self.turn_count+=1
		for current_fsm in self.main_fsm_list:
			if(len(self.main_fsm_list)==0 and self.turn_count > N_food):
				break
			current_fsm.act(self.main_coordinate_system.is_food(current_fsm.pos_x,current_fsm.pos_y,self.vaccine))
			#Phase 1: movement
			if(current_fsm.get_state() == '00'):
				self.main_coordinate_system.move(current_fsm)
				continue
			#Phase 2: food consumption
			if(current_fsm.get_state() == '01'):
				self.main_coordinate_system.food(current_fsm.pos_x,current_fsm.pos_y,-1)
				continue
			#Phase 3: reproduction
			if(current_fsm.get_state() == '10'):
				self.new_fsm_list.append(FSM(current_fsm.pos_x,current_fsm.pos_y))
				continue
			#Phase 4: dying FSMs removed
			if(current_fsm.get_state() == '11'):
				self.main_fsm_list.remove(current_fsm)
				continue
		#Add new FSMs to main list
		if(len(self.new_fsm_list)>0):
			self.main_fsm_list.extend(self.new_fsm_list)
		self.new_fsm_list.clear()
		#Handle susceptibility
		if(self.turn_count<N_food):
			self.main_coordinate_system.generate_food()
		if(self.turn_count>N_food and self.turn_count<N_food2):
			self.main_coordinate_system.remove_food()

	def insert_fsm(self,x,y):
		self.main_fsm_list.append(fsm.FSM(x,y))

In [6]:
#import operate
from numpy import array,ndindex,zeros
#import constants

class Report():

	def __init__(self,oper,title,title2,times):
		self.main_operater = oper
		self.food_count = []
		self.turn_count = []
		self.virus_count = []
		self.output_title = title
		self.output_title2 = title2
		self.output_flag = True
		self.time_points = [N_turns*i for i in times]

	def update(self):
		self.virus_count.append(len(self.main_operater.main_fsm_list))
		self.turn_count.append(self.main_operater.turn_count)
		self.food_count.append(sum(sum(self.main_operater.main_coordinate_system.space)))
		if(max(self.turn_count) == 100):
			if(self.virus_count[-1]==0):
				self.output_flag = False

	def output(self):
		if(self.output_flag):
			fout = open(self.output_title,'w')
			n = max(self.turn_count)
			for i in range(0,n):
				output_string = str(self.turn_count[i])+" "+str(self.virus_count[i])+" "+str(self.food_count[i])+"\n"
				fout.write(output_string)
			fout.close()

	def reveal_virus(self,turn):
		if(turn in self.time_points and self.output_flag):
			fileString = self.output_title2+"_turn"+str(turn)+".dat"
			fout = open(fileString,'w')
			nx = self.main_operater.main_coordinate_system.space.shape[0]
			ny = self.main_operater.main_coordinate_system.space.shape[1]
			virus_array = zeros((nx,ny))
			for temp_fsm in self.main_operater.main_fsm_list :
				virus_array[temp_fsm.pos_x,temp_fsm.pos_y] += 1
			for i in range(0,nx):
				for j in range(0,ny):
					fout.write(str(virus_array[i,j]))
					fout.write(" ")
				fout.write("\n")
			fout.close()

In [9]:
#from coordinate_system import CoordinateSystem
#from fsm import FSM
#from vaccine import Vaccine
#from operate import Operate
#import constants
#from report import Report
from multiprocessing import Process
from random import randint

def plan(fileoutName,virusPlotName,runNumber):

	fileTitle = fileoutName+str(runNumber)+".dat"
	fileTitle2 = virusPlotName+str(runNumber)+".dat"

	l = []
	for i in range(0,N_vacc):
		x = randint(cs_minX,cs_maxX)
		y = randint(cs_minY,cs_maxY)
		l.append((x,y))

	operator = Operate(CoordinateSystem(),[FSM(50,50)],Vaccine(l))
	reporter = Report(operator,fileTitle,fileTitle2,[0,0.1,0.2,0.3,0.4,0.5])

	for i in range(N_turns):
		operator.run()
		reporter.update()
		reporter.reveal_virus(i)

	reporter.output()
    
def main():
    for i in range(0,3):
        plan('Oct31_run_','virusSpatial_Oct31_',i)

if __name__ == "__main__":
	main()